In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from prophet.plot import plot_plotly, plot_components_plotly
from prophet import Prophet
from statsmodels.tools.eval_measures import rmse

df = pd.read_csv('/Users/alanchan/Documents/UCL/Year 4/ELEC0088/SNS assignment 22-23/Tesla Stock Price/01_01_18_to_13_03_23.csv')
print(df.info())
df.dropna(inplace= True)
df.reset_index(drop=True, inplace=True)
print(df.info())

In [ ]:
# print(df.tail())
df = df.rename(columns={'Price': 'y', 'Date':'ds'})
# print(df.tail())
df['ds'] = pd.to_datetime(df['ds'], infer_datetime_format=True)
suffixes = {"M": 1000000, "B": 100000000}
temp = []
for i in df['Vol.']:
    if i[-1] in suffixes.keys():
        temp.append(float(i[:-1]) * suffixes.get(i[-1]))
df['Vol.'] = temp
print(df.tail())

In [ ]:
df.sort_values(by='ds', inplace = True)
df.reset_index(drop=True, inplace=True)
# print(df.head())
print(df.tail())

In [ ]:
df.plot(x='ds',y='y',figsize=(18,6))
print(len(df))

In [ ]:
train = df.iloc[:len(df)-365]
test = df.iloc[len(df)-365:]

print(len(train))
print(len(test))

print(test.head())

In [ ]:
def preprocessing (multi_var,periods):
    t = train[['ds',multi_var]]
    t = t.rename(columns={multi_var: 'y', 'date':'ds'})
    model = Prophet()
    model.fit(t)
    future = model.make_future_dataframe(periods=periods)
    forecast = model.predict(future)
    return forecast['yhat']

periods = 365*2

Open = preprocessing('Open',periods)
High = preprocessing('High',periods)
Low = preprocessing('Low',periods)
Vol = preprocessing('Vol.',periods)

In [ ]:
m = Prophet()
m.add_regressor('Open')
m.add_regressor('High')
m.add_regressor('Low')
m.add_regressor('Vol.')

m.fit(train)
future = m.make_future_dataframe(periods=periods) #MS for monthly, H for hourly
future['Open'] = Open
future['High'] = High
future['Low'] = Low
future['Vol.'] = Vol

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
plot_plotly(m ,forecast)

In [ ]:
plot_components_plotly(m, forecast)

In [ ]:
predictions = forecast.iloc[len(train):len(train)+len(test)]['yhat']
print("Root Mean Squared Error between actual and  predicted values: ",rmse(predictions,test['y']))
print("Mean Value of Test Dataset:", test['y'].mean())

In [ ]:
training = forecast.iloc[:len(train)]['yhat']
print("Root Mean Squared Error between actual and  predicted values: ",rmse(training,train['y']))